In [1]:
# !pip install "ray[tune]" optuna

In [2]:
# pip install -U ipywidgets

In [3]:
# !/usr/bin/python3 -m pip install virtualenv

In [4]:
# !apt-get install libcudnn8=8.6.* 

In [5]:
import json
import os
import tensorflow as tf
import gc
# Train final model with best parameters
from lstm_trainer import SegmentationTrainer
from lstm_tuner import LSTMTuner


2025-01-02 23:53:19.388272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 23:53:20.187258: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
# Function to run a complete experiment
def run_experiment(config_name, data_length_tune=40000, data_length_train=200000):
    config = {
        "model_name": config_name,
        "data_path": "/home/docker/data/work_projects/Segmentation-Models/data/valid_linearizations_4.csv",
        "save_path": "models_200_finals_100",
        "data_length": data_length_tune, 
        "label_smoothing": 0.1, 
        "clip_norm": 1.0,
        "latent_dim": 256,
        "batch_size": 64,
        "num_epochs": 60,
        "train_ratio": 0.8,
        "validation_ratio": 0.1,
        "test_ratio": 0.1,
        "random_seed": 20,
        "initial_lr": 0.00001,
        "decay_steps": 10000,
        "decay_rate": 0.9,
        "dropout_rate": 0.1,
        "patience": 5,
        "validation_split": 0.1
    }
    
    # Setup directories
    save_dir = os.path.join(os.getcwd(), config["save_path"], config["model_name"])
    os.makedirs(save_dir, exist_ok=True)
    config_path = os.path.join(save_dir, "config.json")
    print(f"\nStarting experiment for {config_name}")
    print(f"Saving config to: {config_path}")
    
    try:
        # Save initial config
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
            
        # Run hyperparameter search
        print(f"Running hyperparameter search for {config_name}")
        tuner = LSTMTuner(config)
        best_config, analysis = tuner.run_hyperparameter_search(
            num_samples=10,
            num_epochs=5
        )
        
        # Update config with best parameters
        print(f"Applying best config and starting training for {config_name}")
        final_config = tuner.apply_best_config(best_config)
        final_config["data_length"] = data_length_train  # Full dataset for training
        final_config["batch_size"] = 64
        with open(config_path, 'w') as f:
            json.dump(final_config, f, indent=2)
        
        # Train final model
        trainer = SegmentationTrainer(config_path)
        model, data_processor = trainer.train()
        
        # Clear GPU memory
        tf.keras.backend.clear_session()
        gc.collect()
        
        print(f"Completed experiment for {config_name}")
        return model, data_processor
        
    except Exception as e:
        print(f"Error in experiment {config_name}: {str(e)}")
        raise
    finally:
        # Cleanup
        tf.keras.backend.clear_session()
        gc.collect()

# Run experiments sequentially
experiments = ["segmenter_one", "segmenter_two", "segmenter_three"]

for exp_name in experiments:
    print(f"\n{'='*50}")
    print(f"Starting experiment: {exp_name}")
    print(f"{'='*50}")
    
    model, data_processor = run_experiment(exp_name)
    
    # Force cleanup between experiments
    tf.keras.backend.clear_session()
    gc.collect()
    
    print(f"\nCompleted experiment: {exp_name}")
    print(f"{'='*50}\n")


Starting experiment: segmenter_one

Starting experiment for segmenter_one
Saving config to: /home/docker/data/work_projects/Segmentation-Models/lstm_model/models_200_finals_100/segmenter_one/config.json
Running hyperparameter search for segmenter_one


2025-01-02 23:53:24,748	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-01-02 23:53:25,881	INFO worker.py:1752 -- Started a local Ray instance.
2025-01-02 23:53:26,800	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2025-01-02 23:53:27,523	WARNING packaging.py:405 -- File /home/docker/data/work_projects/Segmentation-Models/lstm_model/models_200/segmenter_three/lstm_model.h5 is very large (25.57MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/home/docker/data/work_projects/Se

(pid=23352) 2025-01-02 23:53:43.095836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(_training_function pid=23352) Original dataset shape: (367178, 4)
(_training_function pid=23352) Cut-off dataset shape: (40000, 4)


(_training_function pid=23352) WARNING:tensorflow:Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
(_training_function pid=23352) WARNING:tensorflow:Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


...

2250/2250 [==============================] - 202s 90ms/step - loss: 0.1340 - accuracy: 0.9715 - val_loss: 0.1293 - val_accuracy: 0.9722
Epoch 47/60
 544/2250 [======>.......................] - ETA: 2:29 - loss: 0.1340 - accuracy: 0.9715

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2188/2250 [============================>.] - ETA: 5s - loss: 0.1334 - accuracy: 0.9717

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2250/2250 [==============================] - 210s 93ms/step - loss: 0.3719 - accuracy: 0.9611 - val_loss: 0.3558 - val_accuracy: 0.9652
Epoch 12/60
2250/2250 [==============================] - 212s 94ms/step - loss: 0.3635 - accuracy: 0.9633 - val_loss: 0.3502 - val_accuracy: 0.9663
Epoch 13/60
2250/2250 [==============================] - 205s 91ms/step - loss: 0.3578 - accuracy: 0.9647 - val_loss: 0.3466 - val_accuracy: 0.9668
Epoch 14/60
2250/2250 [==============================] - 207s 92ms/step - loss: 0.3533 - accuracy: 0.9656 - val_loss: 0.3436 - val_accuracy: 0.9674
Epoch 15/60
2250/2250 [==============================] - 211s 94ms/step - loss: 0.3493 - accuracy: 0.9665 - val_loss: 0.3394 - val_accuracy: 0.9682
Epoch 16/60
2250/2250 [==============================] - 206s 92ms/step - loss: 0.3460 - accuracy: 0.9672 - val_loss: 0.3384 - val_accuracy: 0.9683
Epoch 17/60
2250/2250 [==============================] - 210s 94ms/step - loss: 0.3430 - accuracy: 0.9678 - val_loss: 0.3368